In [1]:
import os

In [2]:
dataset_path = "/kaggle/input/mesiodens-dataset"

In [3]:
# Define paths for images and labels
images_dir = os.path.join(dataset_path, "Original Images")
labels_dir = os.path.join(dataset_path, "YOLO Format")

In [4]:
# Get list of image and label files
image_files = sorted([os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
label_files = sorted([os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith('.txt')])

In [5]:
# Print some samples
print("Sample image files:", image_files[:5])
print("Sample label files:", label_files[:5])

Sample image files: ['/kaggle/input/mesiodens-dataset/Original Images/Aayan Ishfaq(1).jpg', '/kaggle/input/mesiodens-dataset/Original Images/Abhay Josh(2).jpg', '/kaggle/input/mesiodens-dataset/Original Images/Abhijot Opg(3).png', '/kaggle/input/mesiodens-dataset/Original Images/Abhinav(4).jpg', '/kaggle/input/mesiodens-dataset/Original Images/Adhiraj Singh Opg(5).jpg']
Sample label files: ['/kaggle/input/mesiodens-dataset/YOLO Format/Aayan Ishfaq(1).txt', '/kaggle/input/mesiodens-dataset/YOLO Format/Abhay Josh(2).txt', '/kaggle/input/mesiodens-dataset/YOLO Format/Abhijot Opg(3).txt', '/kaggle/input/mesiodens-dataset/YOLO Format/Abhinav(4).txt', '/kaggle/input/mesiodens-dataset/YOLO Format/Adhiraj Singh Opg(5).txt']


In [6]:
# Define working directory
dest_images_dir = "/kaggle/working/images"
dest_labels_dir = "/kaggle/working/labels"
data_yaml_path = "/kaggle/working/data.yaml"

In [7]:
train_dir = "/kaggle/working/train"
val_dir = "/kaggle/working/val"
test_dir = "/kaggle/working/test"

In [8]:
# Create directories if they don't exist
os.makedirs(dest_images_dir, exist_ok=True)
os.makedirs(dest_labels_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [9]:
import shutil

# Copy images and labels to working directory
for file in os.listdir(images_dir):
    shutil.copy(os.path.join(images_dir, file), os.path.join(dest_images_dir, file))
for file in os.listdir(labels_dir):
    shutil.copy(os.path.join(labels_dir, file), os.path.join(dest_labels_dir, file))

In [10]:
# Ensure every image has an annotation file
for img_file in os.listdir(dest_images_dir):
    label_file = os.path.join(dest_labels_dir, img_file.rsplit(".", 1)[0] + ".txt")
    if not os.path.exists(label_file):
        open(label_file, "w").close()  # Create an empty annotation file

In [11]:
# Apply CLAHE to all images
def apply_clahe(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img)
    cv2.imwrite(image_path, img_clahe)

In [12]:
import cv2

for img_file in os.listdir(dest_images_dir):
    apply_clahe(os.path.join(dest_images_dir, img_file))

In [13]:
# Get list of image and label files
def get_image_files():
    return sorted([os.path.join(dest_images_dir, f) for f in os.listdir(dest_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

def get_label_files():
    return sorted([os.path.join(dest_labels_dir, f) for f in os.listdir(dest_labels_dir) if f.endswith('.txt')])

In [14]:
image_files = get_image_files()
label_files = get_label_files()

In [15]:
# Print some samples
print("Sample image files:", image_files[:5])
print("Sample label files:", label_files[:5])

Sample image files: ['/kaggle/working/images/Aayan Ishfaq(1).jpg', '/kaggle/working/images/Abhay Josh(2).jpg', '/kaggle/working/images/Abhijot Opg(3).png', '/kaggle/working/images/Abhinav(4).jpg', '/kaggle/working/images/Adhiraj Singh Opg(5).jpg']
Sample label files: ['/kaggle/working/labels/Aayan Ishfaq(1).txt', '/kaggle/working/labels/Abhay Josh(2).txt', '/kaggle/working/labels/Abhijot Opg(3).txt', '/kaggle/working/labels/Abhinav(4).txt', '/kaggle/working/labels/Adhiraj Singh Opg(5).txt']


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
# Train-test split
train_images, test_images, train_labels, test_labels = train_test_split(image_files, label_files, test_size=0.15, random_state=42)

In [18]:
# Move files to respective directories
for img, lbl in zip(train_images, train_labels):
    shutil.move(img, os.path.join(train_dir, os.path.basename(img)))
    shutil.move(lbl, os.path.join(train_dir, os.path.basename(lbl)))
for img, lbl in zip(test_images, test_labels):
    shutil.move(img, os.path.join(test_dir, os.path.basename(img)))
    shutil.move(lbl, os.path.join(test_dir, os.path.basename(lbl)))

In [19]:
# Create data.yaml file
data_yaml_content = f"""
path: /kaggle/working
train: train
val: test  # Using train as validation for now
test: test
names:
  0: object
"""

In [20]:
with open(data_yaml_path, "w") as f:
    f.write(data_yaml_content)

In [21]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 16.6 MB/s eta 0:00:00a 0:00:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [22]:
# Train YOLO v11n model
yolo_model = YOLO("yolo11n.pt")
yolo_model.train(
    data=data_yaml_path,
    epochs=500,
    imgsz=1280,
    augment=True,  # Enable augmentations
    fliplr=0.5,  # 50% chance of horizontal flipping
    mosaic=0.0,
    hsv_h=0.015,  # Adjust hue (±1.5%)
    hsv_s=0.7,  # Adjust saturation (±70%)
    hsv_v=0.4,  # Adjust brightness (±40%)
)

100%|██████████| 5.35M/5.35M [00:00<00:00, 68.2MB/s]


Ultralytics 8.3.93 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=500, time=None, patience=100, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/train... 67 images, 2 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<00:00, 1008.59it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.84it/s]


                   all         12         13      0.874      0.534      0.621      0.337


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.4ms preprocess, 17.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cfcf4aaac80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [23]:
# Test YOLO model on test set
results = yolo_model.predict(source=test_dir, save=True)
print("Test results generated.")


image 1/12 /kaggle/working/test/Aayan Ishfaq(1).jpg: 672x1280 1 object, 160.9ms
image 2/12 /kaggle/working/test/Adhiraj Singh Opg(5).jpg: 608x1280 (no detections), 133.7ms
image 3/12 /kaggle/working/test/Anshpreet Opg(11).jpg: 672x1280 7 objects, 35.8ms
image 4/12 /kaggle/working/test/Arshdeep Opg(13).jpg: 672x1280 1 object, 26.3ms
image 5/12 /kaggle/working/test/Bhoomik Pre-Treatment(19).jpg: 640x1280 1 object, 138.3ms
image 6/12 /kaggle/working/test/Dilpreet Singh Opg(23).jpg: 640x1280 1 object, 26.4ms
image 7/12 /kaggle/working/test/Gurroop Singh Opg(29).jpg: 576x1280 2 objects, 100.3ms
image 8/12 /kaggle/working/test/Harmanjot(31).jpg: 640x1280 2 objects, 27.3ms
image 9/12 /kaggle/working/test/Harminder(32).jpg: 704x1280 3 objects, 99.0ms
image 10/12 /kaggle/working/test/Harshdeep(34).jpg: 512x1280 4 objects, 130.6ms
image 11/12 /kaggle/working/test/Munsi Ram(50).jpg: 608x1280 (no detections), 27.3ms
image 12/12 /kaggle/working/test/Simran Kaur(71).png: 640x1280 1 object, 28.4ms
S

In [24]:
# Print precision, recall, and IoU metrics
metrics = yolo_model.val()

Ultralytics 8.3.93 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /kaggle/working/test.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


                   all         12         13      0.874      0.534      0.639      0.356


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.4ms preprocess, 30.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train3


In [25]:
print(f"Precision: {metrics.box.mp}")
print(f"Recall: {metrics.box.mr}")

Precision: 0.8740463969857968
Recall: 0.534353710092663


In [26]:
print(f"mAP@50: {metrics.box.map50}")

mAP@50: 0.6392822957465814
